http://www2.cao.ie/points/l8.php

***

In [1]:
#convenient http requests
import requests as rq

# regular expressions
import re

#dates and times
import datetime as dt


In [2]:
# fetch the CAO points URL
# resp= response from the URL
# rq= request library
# get= get the url (http version)
resp = rq.get ('http://www2.cao.ie/points/l8.php')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

<Response [200]>

<br>

# save original data set

In [3]:
#datetime function has a data structure called datetime
#field for time, teat ,month etc
#dt.datetime.now()
now = dt.datetime.now()


In [4]:
print(now)

2021-10-19 16:34:24.735494


### use the current time as a filename - need to remove colons, decimal points etc

In [5]:
# good programmers use only alphanumeric characters ,dash and dot in file names
#create a filepath for the original data
#use year at start to ensure alphabetic sorted when filed
# below creates the csv file and saves it in a new 'data' folder in active location  ie. \repo\Fun_of..
path = 'data/cao2021_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'

In [6]:
# format now as a string
now.strftime('%Y%m%d_%H%M%S')

'20211019_163424'

### the server uses the wrong encoding fix it before saving

In [7]:
#RESP = RESPONSE
#'resp.text' takes the response and converts it to text- but still has original errors
#thus not best way to save THIS original file
#but resp.text is a typical way to save

In [8]:
original_encoding = resp.encoding
resp.encoding = 'cp1252'

In [9]:
#save the original html file

with open(path, 'w') as f:
    f.write(resp.text)    

In [ ]:
resp.text
#\n is a new line (common Unix command)

In [ ]:
#how to loop throught the linesv in request. iterate through the lines in request
for line in resp.iter_lines():
    print(line)

In [12]:
#compile the regular expression for matching lines
# re= regular expression. Each chsracter is a string but together form an instruction.
# r. = 'raw' used in python. It means don't evaluate backslashes in encountered string literals  i.e. \n \t i.e 
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

<br>

### error on the server= way to fix

match only the lines we want -the ones representing courses 
 technically the server says we should decode as:
 
content-type: text/html; charset: iso-8859-1

  but
    -one line uses /x96 which isn't defined in iso-8859-1
    -therefore we used the similiar decoding standard (codec) cp-1252 which is very similar but includes \x96

In [13]:
#for the below if used in assessment should be cleaned up. Check regular expressions are working properly
# if want to save output of this then need a filepath and file format like the original data above

#the filepath for the csv file
path = 'data/cao2021_csv' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'

no_lines = 0
#open the csv file for writing
with open(path, 'w') as f:
   # f.write(resp.text)   
    #loop through the lines of the response content
    for line in resp.iter_lines():
   #decode the line using the encoding not flagged by the response from CAO
        dline = line.decode('cp1252')
        if re_course.fullmatch(dline):
            no_lines = no_lines + 1
            #print(line) no need to print so commented out
            # version 1 using regular expression
            # csv_version = re_course.sub(r'\1,\2,\3,\4', dline) version 1 using re commented out
            #print(csv_version)  no need to print so commented out
            #alternate way than regular expression is to use python in built split function
            # split splits the line
            #(+ ) instructs to do this where two or more spaces
            linesplit = re.split(' +', dline)
            #print(','.join(linesplit))
            #f.write(csv_version + '\n') - saving of old RE way
            #rejoin the substrings with commasin between
            f.write(','.join(linesplit) + '\n')
print(f"total number of lines is {no_lines}.")       

total number of lines is 922.


In [14]:
#old code
#how to loop throught the linesv in request. iterate through the lines in request using a regular expression
#for line in resp.iter_lines():
   # if re.match('[A-Z]{2}[0-9]{3}  .* *(0-9){3} *', line.decode('utf-8') ):
               # print(line)

https://www.prepressure.com/library/technology/ascii-binary-hex

http://www.i18nqa.com/debug/table-iso8859-1-vs-windows-1252.html

https://docs.python.org/3/library/datetime.html

In [15]:
#strftime turns datetime to strings
#strptime turns strings into datetime